In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,1818
2,Huelva,Confirmados PDIA 14 días,537
3,Huelva,Tasa PDIA 14 días,"105,14094172607358"
4,Huelva,Confirmados PDIA 7 días,248
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,4
629,Municipio de Huelva sin especificar,Total Confirmados,27
630,Municipio de Huelva sin especificar,Curados,2


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1818.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  453.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1916 personas en los últimos 7 días 

Un positivo PCR cada 1228 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,1818.0,248.0,537.0,510743.0,48.556711,105.140942,49.0
Huelva-Costa,919.0,124.0,229.0,288115.0,43.038370,79.482151,36.0
Huelva (capital),453.0,75.0,117.0,143663.0,52.205509,81.440594,21.0
Condado-Campiña,632.0,93.0,238.0,155057.0,59.977944,153.491942,11.0
Sierra de Huelva-Andévalo Central,240.0,27.0,65.0,67571.0,39.957970,96.195113,7.0
San Juan del Puerto,71.0,34.0,50.0,9300.0,365.591398,537.634409,5.0
Lepe,136.0,20.0,51.0,27431.0,72.910211,185.921038,4.0
Villarrasa,17.0,7.0,13.0,2176.0,321.691176,597.426471,3.0
Isla Cristina,73.0,5.0,11.0,21264.0,23.513920,51.730625,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Campillo (El),27.0,1.0,25.0,2023.0,49.431537,1235.788433,0.0
Villarrasa,17.0,7.0,13.0,2176.0,321.691176,597.426471,3.0
Villanueva de los Castillejos,18.0,11.0,16.0,2820.0,390.070922,567.375887,2.0
San Juan del Puerto,71.0,34.0,50.0,9300.0,365.591398,537.634409,5.0
Jabugo,14.0,9.0,10.0,2250.0,400.000000,444.444444,3.0
Palma del Condado (La),87.0,12.0,47.0,10761.0,111.513800,436.762383,0.0
Bollullos Par del Condado,106.0,12.0,41.0,14272.0,84.080717,287.275785,0.0
Villablanca,10.0,4.0,8.0,2848.0,140.449438,280.898876,3.0
Lucena del Puerto,14.0,3.0,9.0,3371.0,88.994364,266.983091,0.0
